In [387]:
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import requests
import json
import geocoder
import folium
import sklearn
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors

In [28]:
toronto_wiki_url = "https://en.wikipedia.org/wiki/List_of_neighbourhoods_in_Toronto"
postal_codes_wiki_url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

In [29]:
wiki_html = requests.get(postal_codes_wiki_url)

In [30]:
wiki_soup = BeautifulSoup(wiki_html.text, "lxml")

In [13]:
wiki_soup

<!DOCTYPE html>
<html class="client-nojs" dir="ltr" lang="en">
<head>
<meta charset="utf-8"/>
<title>List of neighbourhoods in Toronto - Wikipedia</title>
<script>document.documentElement.className="client-js";RLCONF={"wgBreakFrames":!1,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgRequestId":"661dad50-4642-4247-84c0-33ba1ec47a05","wgCSPNonce":!1,"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":!1,"wgNamespaceNumber":0,"wgPageName":"List_of_neighbourhoods_in_Toronto","wgTitle":"List of neighbourhoods in Toronto","wgCurRevisionId":1016973688,"wgRevisionId":1016973688,"wgArticleId":1150939,"wgIsArticle":!0,"wgIsRedirect":!1,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Webarchive template wayback links","Articles with short description","Short description is different from W

In [34]:
attrs = {'class': 'wikitable'}

# list_of_tables = wiki_soup.find_all('table', attrs = attrs)
# nb_table = list_of_tables[1]

table = wiki_soup.find('table')

In [35]:
table

<table cellpadding="2" cellspacing="0" rules="all" style="width:100%; border-collapse:collapse; border:1px solid #ccc;">
<tbody><tr>
<td style="width:11%; vertical-align:top; color:#ccc;">
<p><b>M1A</b><br/><span style="font-size:85%;"><i>Not assigned</i></span>
</p>
</td>
<td style="width:11%; vertical-align:top; color:#ccc;">
<p><b>M2A</b><br/><span style="font-size:85%;"><i>Not assigned</i></span>
</p>
</td>
<td style="width:11%; vertical-align:top;">
<p><b>M3A</b><br/><span style="font-size:85%;"><a href="/wiki/North_York" title="North York">North York</a><br/>(<a href="/wiki/Parkwoods" title="Parkwoods">Parkwoods</a>)</span>
</p>
</td>
<td style="width:11%; vertical-align:top;">
<p><b>M4A</b><br/><span style="font-size:85%;"><a href="/wiki/North_York" title="North York">North York</a><br/>(<a href="/wiki/Victoria_Village" title="Victoria Village">Victoria Village</a>)</span>
</p>
</td>
<td style="width:11%; vertical-align:top;">
<p><b>M5A</b><br/><span style="font-size:85%;"><a hr

In [24]:
nb_table.find_all('td')[0:6]

[<td>129
 </td>,
 <td>Agincourt North
 </td>,
 <td>Scarborough
 </td>,
 <td><a href="/wiki/Agincourt,_Toronto" title="Agincourt, Toronto">Agincourt</a> and Brimwood
 </td>,
 <td>
 </td>,
 <td>128
 </td>]

In [27]:
for td in nb_table.find_all('td')[0:6]:
    print(td.text.strip())

129
Agincourt North
Scarborough
Agincourt and Brimwood

128


In [ ]:
table_contents = []

counter = 1
for td in nb_table.find_all('td'):
    cell_content = td.text.strip()
    table_row = {}
    if counter == 1:
        table_row['PostalCode'] = cell_content
    if counter == 2:
        table_row['Borough'] = 

In [43]:
ex_1 = table.find_all('td')[0]

In [44]:
ex_2 = table.find_all('td')[2]

In [72]:
ex_3 = table.find_all('td')[6]

In [45]:
ex_1

<td style="width:11%; vertical-align:top; color:#ccc;">
<p><b>M1A</b><br/><span style="font-size:85%;"><i>Not assigned</i></span>
</p>
</td>

In [46]:
ex_2

<td style="width:11%; vertical-align:top;">
<p><b>M3A</b><br/><span style="font-size:85%;"><a href="/wiki/North_York" title="North York">North York</a><br/>(<a href="/wiki/Parkwoods" title="Parkwoods">Parkwoods</a>)</span>
</p>
</td>

In [48]:
ex_1.find('b')

<b>M1A</b>

In [49]:
ex_1.find('i')

<i>Not assigned</i>

In [50]:
ex_2.find('b')

<b>M3A</b>

In [55]:
ex_2.find_all('a')

[<a href="/wiki/North_York" title="North York">North York</a>,
 <a href="/wiki/Parkwoods" title="Parkwoods">Parkwoods</a>]

In [57]:
ex_2.find_all('a')[1].text

'Parkwoods'

In [73]:
ex_3

<td style="width:11%; vertical-align:top;">
<p><b>M7A</b><br/><span style="font-size:85%;"><a href="/wiki/Queen%27s_Park_(Toronto)" title="Queen's Park (Toronto)">Queen's Park</a><br/>(Ontario Provincial Government)</span>
</p>
</td>

In [80]:
ex_3.span.text.split('(')[1].split(')')

['Ontario Provincial Government', '']

In [102]:
table_contents = []
for cell in table.find_all('td'):
    new_row = {}
    if cell.find('i'):
        pass
    else:
        new_row['PostalCode'] = cell.find('b').text
        tmp_str = cell.span.text.split('(')
        new_row['Borough'] = tmp_str[0]
        new_row['Neighborhood'] = tmp_str[1].split(')')[0].replace(' / ', ', ')
        table_contents.append(new_row)

# print(table_contents)

In [100]:
table.find_all('td')[4].span.text.split('(')[1].split(')')[0].replace(' / ', ', ')

'Regent Park, Harbourfront'

In [103]:
df = pd.DataFrame(table_contents)
df['Borough']=df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})
df

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government
...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East Toronto Business,Enclave of M4L
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


In [106]:
with open('foursquare_secrets.json') as f_in:
    fsq_secrets = json.load(f_in)

In [169]:
CLIENT_SECRET = fsq_secrets['CLIENT_SECRET']
CLIENT_ID = fsq_secrets['CLIENT_ID']
VERSION = '20180605'
LIMIT = 100
radius = 500

In [125]:
postal_code = 'M3A'

coords = None
count = 0

while not coords and count < 100:
#     g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
    g = geocoder.google('Mountain View, CA')
    coords = g.latlng
    count += 1

print(coords)

None


In [124]:
g

<[REQUEST_DENIED] Google - Geocode [empty]>

In [123]:
g = geocoder.google('Mountain View, CA')
g.latlng

In [131]:
df_geospatial = pd.read_csv('Geospatial_Coordinates.csv').rename(columns = {'Postal Code': 'PostalCode'})

In [132]:
df_geospatial.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [128]:
df_geospatial.shape

(103, 3)

In [133]:
df_merged = pd.merge(df, df_geospatial, how = 'inner', on = 'PostalCode')
df_merged.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494


In [420]:
df_merged[df_merged.PostalCode == 'M5A']

,PostalCode,Borough,Neighborhood,Latitude,Longitude
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.65426,-79.360636


In [141]:
df_merged[df_merged.Borough.str.contains('Toronto')].shape

(39, 5)

In [143]:
# g = geocoder.google('Toronto, Canada', key = 'apikey')
g.latlng

In [152]:
tor_lat = 43.6532
tor_lon = -79.3832

In [377]:
df_merged.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494


In [378]:
df_tor_only = df_merged[df_merged.Borough.str.contains('Toronto')].reset_index(drop = True)

In [167]:
# df_tor_only = df_merged[df_merged.Borough.str.contains('Toronto')]

map_toronto = folium.Map(location = [tor_lat, tor_lon], zoom_start = 12)

for code, bor, lat, lng in zip(df_tor_only.PostalCode, 
                               df_tor_only.Borough, 
                               df_tor_only.Latitude, 
                               df_tor_only.Longitude):

# for code, bor, lat, lng in zip(df_merged.PostalCode,
#                                df_merged.Borough,
#                                df_merged.Latitude,
#                                df_merged.Longitude):

    folium.CircleMarker(
        [lat, lng],
        radius = 5,
        popup = 'code',
        color = 'blue',
        fill = True).add_to(map_toronto)

In [168]:
map_toronto

In [233]:
df_tor_only[df_tor_only.PostalCode == 'M4M']['PostalCode'].loc[54]

'M4M'

In [234]:
lat = df_tor_only[df_tor_only.PostalCode == 'M4M']['Latitude'].loc[54]
lon = df_tor_only[df_tor_only.PostalCode == 'M4M']['Longitude'].loc[54]

In [235]:
base_url = "https://api.foursquare.com/v2/venues/explore"

params = {
    'client_id': CLIENT_ID,
    'client_secret': CLIENT_SECRET,
    'v': VERSION,
    'll': str(lat) + ',' + str(lon),
    'radius': radius,
    'limit': LIMIT
}

In [236]:
m4m_query = requests.get(base_url, params = params)

In [222]:
m5g_query.json()['response']['groups'][0]['items'][3].keys()

dict_keys(['reasons', 'venue', 'referralId'])

In [223]:
for x in m5g_query.json()['response']['groups'][0]['items']:
    if 'venue' not in x:
        print(x.keys())

In [238]:
df_response = pd.json_normalize(m4m_query.json()['response']['groups'][0]['items'])

In [308]:
df_filtered = df_response[['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']]

In [270]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [285]:
df_filtered.loc[0, 'venue.categories'][0]['name']
# df_filtered.loc[0, 'venue.categories']

'Ice Cream Shop'

In [268]:
for item in m4m_query.json()['response']['groups'][0]['items']:
    if len(item['venue']['categories']) != 1:
        print(venue.name, len(item['venue']['categories']))

In [309]:
df_filtered.head()

,venue.name,venue.categories,venue.location.lat,venue.location.lng
0,Ed's Real Scoop,"[{'id': '4bf58dd8d48988d1c9941735', 'name': 'I...",43.660656,-79.342019
1,Mercury Espresso Bar,"[{'id': '4bf58dd8d48988d1e0931735', 'name': 'C...",43.660806,-79.341241
2,Queen Books,"[{'id': '4bf58dd8d48988d114951735', 'name': 'B...",43.660651,-79.342267
3,Te Aro,"[{'id': '4bf58dd8d48988d1e0931735', 'name': 'C...",43.661373,-79.338577
4,Reliable Halibut and Chips,"[{'id': '4bf58dd8d48988d1ce941735', 'name': 'S...",43.660874,-79.340938


In [293]:
# df_filtered['Categories'] = df_filtered.apply(get_category_type, axis=1)
new_series_tmp = df_filtered['venue.categories'].map(lambda x: x[0]['name'])

In [299]:
new_series_tmp

0                Ice Cream Shop
1                   Coffee Shop
2                     Bookstore
3                   Coffee Shop
4            Seafood Restaurant
5                   Fish Market
6                     Pet Store
7                        Bakery
8                          Café
9                       Gay Bar
10                 Neighborhood
11           Italian Restaurant
12                  Cheese Shop
13          American Restaurant
14              Thai Restaurant
15                      Brewery
16      Comfort Food Restaurant
17    Middle Eastern Restaurant
18             Stationery Store
19                         Café
20                  Coffee Shop
21                       Bakery
22              Coworking Space
23                         Park
24                  Yoga Studio
25    Latin American Restaurant
26          American Restaurant
27                    Gastropub
28         Gym / Fitness Center
29                         Bank
30                          Bar
31      

In [298]:
df_filtered['Categories'] = new_series_tmp

/Users/bridgettelafaye/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [310]:
df_added = df_filtered.assign(Categories = df_filtered['venue.categories'].map(lambda x: x[0]['name']))

In [311]:
df_filtered.head()

,venue.name,venue.categories,venue.location.lat,venue.location.lng
0,Ed's Real Scoop,"[{'id': '4bf58dd8d48988d1c9941735', 'name': 'I...",43.660656,-79.342019
1,Mercury Espresso Bar,"[{'id': '4bf58dd8d48988d1e0931735', 'name': 'C...",43.660806,-79.341241
2,Queen Books,"[{'id': '4bf58dd8d48988d114951735', 'name': 'B...",43.660651,-79.342267
3,Te Aro,"[{'id': '4bf58dd8d48988d1e0931735', 'name': 'C...",43.661373,-79.338577
4,Reliable Halibut and Chips,"[{'id': '4bf58dd8d48988d1ce941735', 'name': 'S...",43.660874,-79.340938


In [312]:
df_added.head()

,venue.name,venue.categories,venue.location.lat,venue.location.lng,Categories
0,Ed's Real Scoop,"[{'id': '4bf58dd8d48988d1c9941735', 'name': 'I...",43.660656,-79.342019,Ice Cream Shop
1,Mercury Espresso Bar,"[{'id': '4bf58dd8d48988d1e0931735', 'name': 'C...",43.660806,-79.341241,Coffee Shop
2,Queen Books,"[{'id': '4bf58dd8d48988d114951735', 'name': 'B...",43.660651,-79.342267,Bookstore
3,Te Aro,"[{'id': '4bf58dd8d48988d1e0931735', 'name': 'C...",43.661373,-79.338577,Coffee Shop
4,Reliable Halibut and Chips,"[{'id': '4bf58dd8d48988d1ce941735', 'name': 'S...",43.660874,-79.340938,Seafood Restaurant


In [313]:
df_final = df_added[['venue.name', 'Categories', 'venue.location.lat', 'venue.location.lng']].rename(columns = {
    'venue.name': 'name',
    'venue.location.lat': 'latitude',
    'venue.location.lng': 'longitude'
})

In [314]:
df_final.head()

,name,Categories,latitude,longitude
0,Ed's Real Scoop,Ice Cream Shop,43.660656,-79.342019
1,Mercury Espresso Bar,Coffee Shop,43.660806,-79.341241
2,Queen Books,Bookstore,43.660651,-79.342267
3,Te Aro,Coffee Shop,43.661373,-79.338577
4,Reliable Halibut and Chips,Seafood Restaurant,43.660874,-79.340938


In [315]:
df_final.shape

(37, 4)

In [323]:
def get_nearby_venues(codes, lats, lons, radius = 500):
    
    count = 0
    
    params = {
        'client_id': CLIENT_ID,
        'client_secret': CLIENT_SECRET,
        'v': VERSION,
        'll': '',
        'radius': radius,
        'limit': LIMIT
    }
    
    results = []
    
    for code, lat, lon in zip(codes, lats, lons):
        
        params['ll'] = str(lat) + ',' + str(lon)
        query = requests.get(base_url, params = params)
        count += 1
        query_items = query.json()['response']['groups'][0]['items']
        for q_item in query_items:
            new_row = [
                code,
                lat,
                lon,
                q_item['venue']['name'],
                q_item['venue']['categories'][0]['name'],
                q_item['venue']['location']['lat'],
                q_item['venue']['location']['lng']
            ]
            results.append(new_row)
        
    df_results = pd.DataFrame(results, columns = [
        'nbhood_name',
        'nbhood_lat',
        'nbhood_lon',
        'venue_name',
        'categories',
        'venue_lat',
        'venue_lon'
    ])
    
    print(count)
    return df_results

In [324]:
tor_nearby = get_nearby_venues(
    df_tor_only['PostalCode'],
    df_tor_only['Latitude'],
    df_tor_only['Longitude']
)

39


In [327]:
tor_nearby.head()

,nbhood_name,nbhood_lat,nbhood_lon,venue_name,categories,venue_lat,venue_lon
0,M5A,43.65426,-79.360636,Roselle Desserts,Bakery,43.653447,-79.362017
1,M5A,43.65426,-79.360636,Tandem Coffee,Coffee Shop,43.653559,-79.361809
2,M5A,43.65426,-79.360636,Cooper Koo Family YMCA,Distribution Center,43.653249,-79.358008
3,M5A,43.65426,-79.360636,Body Blitz Spa East,Spa,43.654735,-79.359874
4,M5A,43.65426,-79.360636,Impact Kitchen,Restaurant,43.656369,-79.356980


In [331]:
len(tor_nearby['categories'].unique())

234

In [340]:
tor_one_hot = pd.get_dummies(tor_nearby[['categories']], prefix = "", prefix_sep = "")

In [333]:
tor_one_hot.shape

(1575, 234)

In [341]:
tor_one_hot = pd.concat([tor_nearby[['nbhood_name']], tor_one_hot], axis = 1)

In [342]:
tor_one_hot.head()

,nbhood_name,Adult Boutique,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Yoga Studio
0,M5A,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,M5A,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,M5A,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,M5A,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,M5A,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [343]:
tor_grouped = tor_one_hot.groupby('nbhood_name').mean().reset_index()

In [344]:
tor_grouped.head()

,nbhood_name,Adult Boutique,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Yoga Studio
0,M4E,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.00000,0.0,0.20000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
1,M4J,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.00000,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
2,M4K,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.023810,0.0,...,0.02381,0.0,0.02381,0.0,0.0,0.0,0.0,0.0,0.0,0.023810
3,M4L,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.00000,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
4,M4M,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.054054,0.0,...,0.00000,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.027027


In [350]:
tor_grouped[['Neighborhood']].head()

,Neighborhood
0,0.200000
1,0.000000
2,0.000000
3,0.000000
4,0.027027


In [345]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [389]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = tor_grouped['nbhood_name']

for ind in np.arange(tor_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(tor_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,Neighborhood,Asian Restaurant,Health Food Store,Trail,Pub,Doner Restaurant,Discount Store,Distribution Center,Dive Bar,Dog Run
1,M4J,Park,Coffee Shop,Convenience Store,Yoga Studio,Diner,Ethiopian Restaurant,Escape Room,Electronics Store,Eastern European Restaurant,Dumpling Restaurant
2,M4K,Greek Restaurant,Coffee Shop,Italian Restaurant,Ice Cream Shop,Furniture / Home Store,Bubble Tea Shop,Indian Restaurant,Spa,Japanese Restaurant,Bookstore
3,M4L,Park,Sandwich Place,Fast Food Restaurant,Board Shop,Movie Theater,Fish & Chips Shop,Sushi Restaurant,Ice Cream Shop,Liquor Store,Italian Restaurant
4,M4M,Coffee Shop,American Restaurant,Bakery,Brewery,Café,Gastropub,Yoga Studio,Dive Bar,Park,Neighborhood


In [354]:
tor_grouped.head()

,nbhood_name,Adult Boutique,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Yoga Studio
0,M4E,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.00000,0.0,0.20000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
1,M4J,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.00000,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
2,M4K,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.023810,0.0,...,0.02381,0.0,0.02381,0.0,0.0,0.0,0.0,0.0,0.0,0.023810
3,M4L,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.00000,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
4,M4M,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.054054,0.0,...,0.00000,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.027027


In [355]:
tor_grouped.drop('nbhood_name', 1).head()

,Adult Boutique,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,...,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Yoga Studio
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.00000,0.0,0.20000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.00000,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.023810,0.0,0.0,...,0.02381,0.0,0.02381,0.0,0.0,0.0,0.0,0.0,0.0,0.023810
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,...,0.00000,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.054054,0.0,0.0,...,0.00000,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.027027


In [359]:
tor_clustering = tor_grouped.drop(columns = ['nbhood_name'])

In [416]:
kclusters = 2
kmeans = KMeans(n_clusters = kclusters, random_state=0).fit(tor_clustering)

In [417]:
# neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
w_cluster_labels = pd.concat([pd.DataFrame({'Cluster Labels': kmeans.labels_}), neighborhoods_venues_sorted], axis = 1)

In [402]:
# neighborhoods_venues_sorted.head()
w_cluster_labels.head()

,Cluster Labels,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,0,M4E,Neighborhood,Asian Restaurant,Health Food Store,Trail,Pub,Doner Restaurant,Discount Store,Distribution Center,Dive Bar,Dog Run
1,2,M4J,Park,Coffee Shop,Convenience Store,Yoga Studio,Diner,Ethiopian Restaurant,Escape Room,Electronics Store,Eastern European Restaurant,Dumpling Restaurant
2,0,M4K,Greek Restaurant,Coffee Shop,Italian Restaurant,Ice Cream Shop,Furniture / Home Store,Bubble Tea Shop,Indian Restaurant,Spa,Japanese Restaurant,Bookstore
3,0,M4L,Park,Sandwich Place,Fast Food Restaurant,Board Shop,Movie Theater,Fish & Chips Shop,Sushi Restaurant,Ice Cream Shop,Liquor Store,Italian Restaurant
4,0,M4M,Coffee Shop,American Restaurant,Bakery,Brewery,Café,Gastropub,Yoga Studio,Dive Bar,Park,Neighborhood


In [379]:
df_tor_only.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
1,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
2,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
3,M4E,East Toronto,The Beaches,43.676357,-79.293031
4,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306


In [418]:
cluster_merged = pd.merge(df_tor_only, w_cluster_labels.set_index('Neighborhood'), left_on = 'PostalCode', right_on = 'Neighborhood')

In [397]:
cluster_merged.shape

(39, 16)

In [398]:
cluster_merged.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,0,Coffee Shop,Pub,Bakery,Park,Café,Breakfast Spot,Theater,Yoga Studio,Event Space,Restaurant
1,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937,0,Clothing Store,Coffee Shop,Bubble Tea Shop,Café,Middle Eastern Restaurant,Japanese Restaurant,Cosmetics Shop,Lingerie Store,Hotel,Ramen Restaurant
2,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,0,Coffee Shop,Café,Cosmetics Shop,Hotel,Cocktail Bar,Park,Restaurant,Clothing Store,Moroccan Restaurant,Creperie
3,M4E,East Toronto,The Beaches,43.676357,-79.293031,0,Neighborhood,Asian Restaurant,Health Food Store,Trail,Pub,Doner Restaurant,Discount Store,Distribution Center,Dive Bar,Dog Run
4,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306,0,Coffee Shop,Cocktail Bar,Bakery,Restaurant,Cheese Shop,Beer Bar,Farmers Market,Pharmacy,Seafood Restaurant,Japanese Restaurant


In [419]:
cluster_merged.groupby('Cluster Labels').count()[['PostalCode']]

,PostalCode
Cluster Labels,
0,3
1,36


In [408]:
map_clusters = folium.Map(location = [tor_lat, tor_lon], zoom_start = 12)
 
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]

colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

markers_colors = []
for lat, lon, code, cluster in zip(cluster_merged['Latitude'], cluster_merged['Longitude'], cluster_merged['PostalCode'], cluster_merged['Cluster Labels']):
    label = folium.Popup(code + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters